In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import itertools
import time
import os
from tqdm import tqdm

# ================================================================
#  LOAD & CLEAN
# ================================================================
BASE = "input/"
data_df = pd.read_csv(os.path.join(BASE, "data_year_team_franchise.csv"))

# Drop obvious leakage
leakage = ['yearID', 'year_label', 'decade_label', 'win_bins']
data_df = data_df.drop(columns=[c for c in leakage if c in data_df.columns], errors='ignore')
data_df = data_df.reset_index(drop=True)

# ================================================================
#  BASE COLUMNS — only raw safe columns (no RD, pythag, etc.)
# ================================================================
base_cols = [
    'G', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'SB',
    'RA', 'ER', 'ERA', 'CG', 'SHO', 'SV', 'IPouts', 'HA', 'HRA',
    'BBA', 'SOA', 'E', 'DP', 'FP', 'mlb_rpg'
]


base_cols = [c for c in base_cols if c in data_df.columns]

data_df['pythag_classic'] = 162 * (data_df['R'] ** 2) / (data_df['R'] ** 2 + data_df['RA'] ** 2 + 1e-8)

# Modern exponent (more accurate, ~1.83–1.91, use 1.83 here)
data_df['pythag_modern'] = 162 * (data_df['R'] ** 1.83) / (data_df['R'] ** 1.83 + data_df['RA'] ** 1.83 + 1e-8)

print("Added pythag_classic and pythag_modern")
print(data_df[['R', 'RA', 'pythag_classic', 'pythag_modern']].head(5))


baseline_features = base_cols + ['pythag_modern']


# ================================================================
#  SAFE CANDIDATES FOR GROUPBY / TARGET ENCODING
# ================================================================
# Only columns that are already categorical or that we bin first
candidates = ['G', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'SB',
    'RA', 'ER', 'ERA', 'CG', 'SHO', 'SV', 'IPouts', 'HA', 'HRA',
    'BBA', 'SOA', 'E', 'DP', 'FP', 'mlb_rpg', 'franchID']

# Add franchise ID (most important categorical)
if 'franchID' in data_df.columns:
    data_df['franchID'] = data_df['franchID'].astype('category')
    candidates.append('franchID')

# Add era and decade indicators (they are binary/categorical)
era_cols = [c for c in data_df.columns if c.startswith('era_')]
decade_cols = [c for c in data_df.columns if c.startswith('decade_')]
candidates.extend(era_cols)
candidates.extend(decade_cols)

# Bin continuous columns that are worth grouping on
print("Binning important continuous columns...")
for col in ['ERA', 'mlb_rpg']:
    if col in data_df.columns:
        try:
            data_df[f'{col}_bin'] = pd.qcut(
                data_df[col].rank(pct=True),  # safer than raw values
                q=10,
                duplicates='drop',
                labels=False
            ).astype('category')
            candidates.append(f'{col}_bin')
            print(f"  Added {col}_bin")
        except Exception as e:
            print(f"Binning {col} failed: {e}")

print("Safe candidates for combinations:", candidates)
print(f"→ {len(candidates)} columns")

# If no candidates left → fallback
if not candidates:
    print("No safe candidates! Using only franchID if present.")
    if 'franchID' in data_df.columns:
        candidates = ['franchID']

# ──── FAST CV EVALUATION ────────────────────────────────────────────
def quick_cv_mae(df, features, target='W', n_estimators=400, lr=0.08, depth=4):
    features = [f for f in features if f in df.columns]
    if not features:
        return np.inf  # bad case

    X = df[features]
    y = df[target]
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    maes = []

    for tr_idx, va_idx in kf.split(X):
        model = XGBRegressor(
            n_estimators=n_estimators,
            learning_rate=lr,
            max_depth=depth,
            subsample=0.85,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=-1,
            verbosity=0
        )
        model.fit(X.iloc[tr_idx], y.iloc[tr_idx])
        pred = model.predict(X.iloc[va_idx])
        maes.append(mean_absolute_error(y.iloc[va_idx], pred))

    return np.mean(maes)

# ================================================================
#  TARGET ENCODING FUNCTION (already good — small tweak)
# ================================================================
def safe_target_encode(df, cols, target='W', folds=5, smooth=20.0):
    missing = [c for c in cols + [target] if c not in df.columns]
    if missing:
        raise KeyError(f"Missing columns: {missing}")

    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    te_values = np.full(len(df), np.nan)
    global_mean = df[target].mean()

    for train_idx, val_idx in kf.split(df):
        train_df = df.iloc[train_idx].copy()
        val_df   = df.iloc[val_idx].copy()

        grp = train_df.groupby(cols, observed=True)[target].agg(
            ['mean', 'count']
        ).reset_index()
        grp['smoothed'] = (grp['mean'] * grp['count'] + global_mean * smooth) / (grp['count'] + smooth)

        merged = val_df[cols].merge(
            grp[cols + ['smoothed']],
            on=cols,
            how='left'
        )
        te_values[val_idx] = merged['smoothed'].fillna(global_mean).values

    return pd.Series(te_values, index=df.index, name=f"TE_{'_'.join(cols)}")

# ================================================================
#  BASELINE (raw columns only)
# ================================================================
baseline_features = base_cols.copy()
# Add the pythag columns you created

if 'pythag_modern' in data_df.columns:
    baseline_features.append('pythag_modern')
if 'pythag_classic' in data_df.columns:
    baseline_features.append('pythag_classic')
    

print("Baseline features now include:", baseline_features)
baseline_mae = quick_cv_mae(data_df, baseline_features)
print(f"Baseline MAE (raw columns): {baseline_mae:.4f}")

# ================================================================
#  ADD SINGLE TE/CE ON FRANCHID + ERA/DECADE (very strong!)
# ================================================================
for col in ['franchID'] + era_cols + decade_cols:
    if col in data_df.columns:
        te = safe_target_encode(data_df, [col], smooth=20)
        data_df[te.name] = te
        print(f"Added single TE: {te.name}")

        # Count encode
        ce_name = f"CE_{col}"
        counts = data_df.groupby(col).size() / len(data_df)
        data_df[ce_name] = data_df[col].map(counts).fillna(0)
        print(f"Added CE: {ce_name}")

# ================================================================
#  COMBINATION SEARCH — only on safe candidates
# ================================================================
if len(candidates) >= 2:
    all_pairs = list(itertools.combinations(candidates, 2))
    print(f"\nTesting {len(all_pairs)} safe 2-way combinations...")

    accepted_features = []
    improvements = []

    for comb in tqdm(all_pairs):
        try:
            te_series = safe_target_encode(data_df, list(comb), smooth=10.0)
            temp_df = data_df.copy()
            temp_df[te_series.name] = te_series

            trial_features = baseline_features + [te_series.name]
            new_mae = quick_cv_mae(temp_df, trial_features)
            delta = baseline_mae - new_mae

           # print(f"{str(comb):35} → Δ={delta:+.4f}  new={new_mae:.4f}")

            if delta > 0.005:  # stricter threshold now
                accepted_features.append(te_series.name)
                improvements.append((te_series.name, delta, new_mae))

        except Exception as e:
            print(f"ERROR {comb}: {type(e).__name__} → {str(e)}")

    # Show results
    if improvements:
        improvements.sort(key=lambda x: x[1], reverse=True)
        print("\nTop improvements:")
        for n, d, m in improvements[:12]:
            print(f"{n:40} Δ={d:+.5f}  MAE={m:.4f}")
    else:
        print("No strong combinations found.")
else:
    print("Not enough safe candidates to form 2-way combinations.")

Added pythag_classic and pythag_modern
     R   RA  pythag_classic  pythag_modern
0  718  732       79.436008      79.568918
1  835  751       89.556075      88.833562
2  768  707       87.688222      87.121991
3  713  598       95.102016      93.924738
4  928  921       81.613296      81.561167
Binning important continuous columns...
  Added ERA_bin
  Added mlb_rpg_bin
Safe candidates for combinations: ['G', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'SB', 'RA', 'ER', 'ERA', 'CG', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'SOA', 'E', 'DP', 'FP', 'mlb_rpg', 'franchID', 'franchID', 'era_1', 'era_2', 'era_3', 'era_4', 'era_5', 'era_6', 'era_7', 'era_8', 'decade_1910', 'decade_1920', 'decade_1930', 'decade_1940', 'decade_1950', 'decade_1960', 'decade_1970', 'decade_1980', 'decade_1990', 'decade_2000', 'decade_2010', 'ERA_bin', 'mlb_rpg_bin']
→ 48 columns
Baseline features now include: ['G', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'SB', 'RA', 'ER', 'ERA', 'CG', 'SHO', 'SV', 'IPou

/tmp/ipykernel_18628/2923174209.py:175: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  counts = data_df.groupby(col).size() / len(data_df)


Added single TE: TE_era_7
Added CE: CE_era_7
Added single TE: TE_era_8
Added CE: CE_era_8
Added single TE: TE_decade_1910
Added CE: CE_decade_1910
Added single TE: TE_decade_1920
Added CE: CE_decade_1920
Added single TE: TE_decade_1930
Added CE: CE_decade_1930
Added single TE: TE_decade_1940
Added CE: CE_decade_1940
Added single TE: TE_decade_1950
Added CE: CE_decade_1950
Added single TE: TE_decade_1960
Added CE: CE_decade_1960
Added single TE: TE_decade_1970
Added CE: CE_decade_1970
Added single TE: TE_decade_1980
Added CE: CE_decade_1980
Added single TE: TE_decade_1990
Added CE: CE_decade_1990
Added single TE: TE_decade_2000
Added CE: CE_decade_2000
Added single TE: TE_decade_2010
Added CE: CE_decade_2010

Testing 1128 safe 2-way combinations...


 78%|███████▊  | 875/1128 [2:43:13<11:40,  2.77s/it]      

ERROR ('franchID', 'franchID'): ValueError → cannot insert franchID, already exists


100%|██████████| 1128/1128 [2:52:39<00:00,  9.18s/it]


Top improvements:
TE_3B_decade_1950                        Δ=+0.02718  MAE=2.9999
TE_BB_era_8                              Δ=+0.02543  MAE=3.0017
TE_BB_decade_2010                        Δ=+0.02543  MAE=3.0017
TE_decade_1960_ERA_bin                   Δ=+0.02435  MAE=3.0028
TE_SO_E                                  Δ=+0.02366  MAE=3.0035
TE_E_decade_1980                         Δ=+0.02318  MAE=3.0039
TE_SHO_HRA                               Δ=+0.02296  MAE=3.0042
TE_IPouts_HRA                            Δ=+0.02290  MAE=3.0042
TE_BBA_FP                                Δ=+0.02220  MAE=3.0049
TE_franchID_era_4                        Δ=+0.01719  MAE=3.0099
TE_franchID_era_4                        Δ=+0.01719  MAE=3.0099
TE_3B_decade_1920                        Δ=+0.01504  MAE=3.0121


In [25]:
data_df['pythag_modern'] = 162 * (data_df['R'] ** 1.83) / (data_df['R'] ** 1.83 + data_df['RA'] ** 1.83 + 1e-8)

# Baseline without pythag
mae_no_pythag = quick_cv_mae(data_df, base_cols)
print(f"MAE without pythag: {mae_no_pythag:.4f}")

# Baseline with pythag
mae_with_pythag = quick_cv_mae(data_df, base_cols + ['pythag_modern'])
print(f"MAE with pythag_modern: {mae_with_pythag:.4f}")

# Improvement
delta = mae_no_pythag - mae_with_pythag
print(f"Improvement from adding pythag: {delta:+.4f} MAE")

MAE without pythag: 3.1331
MAE with pythag_modern: 3.0550
Improvement from adding pythag: +0.0781 MAE
